OBS: Algumas partes foram comentadas para poder rodar sem a presença do arduino
--> Buscar essas partes por "BACALHAU"

IMPORTAÇÃO DE BIBLIOTECAS

In [1]:
import cv2
import mediapipe as mp
import math
import pandas as pd
from datetime import datetime 
from tkinter import *
from tkinter import ttk, filedialog
import pygame
import os
#from envia_serial import * #BACALHAU
import json
import threading
import cv2
import mediapipe as mp
import tkinter as tk
from tkinter import ttk, filedialog
import pygame
import os
import json


pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


INICIALIZAÇÃO DO ARDUINO

In [2]:
#arduino = serial.Serial(port='COM5', baudrate=9600, timeout=1)  
#BACALHAU

## FUNÇÕES

FUNÇÕES PARA CALCULAR ANGULO

Calculo do angulo em relação a uma reta

In [3]:
def calcula_angulo(a_x: float, a_y: float, b_x: float, b_y: float):
    '''
    Função que calcula angulo entre um vetor e um vetor vertical a partir de 2 pontos.

    Calculo efetuado a partir da seguinte fórmula: cos θ = (u.v)/(|u||v|)

    O ponto (a) deve ser o ponto mais baixo, que se mexe em relação a referência
    O ponto (b) deve ser o ponto de origem (o ponto mais alto, que não mexe)

    Exemplo: Ponto (a) é o mão direita (ponto de refêrencia móvel) e ponto (b) é o ombro direito (ponto imóvel)
    '''

    # Calculo dos vetores
    vetor_ab = (b_x - a_x, b_y - a_y) # Tupla com coordenadas do vetor
    vetor_vertical = (0,-1) # Vetor (0,-1) pois preciso pegar o ângulo entre a reta (passando pelos dois pontos) e uma reta vertical

    # Calculo dos componentes da formula
    produto_escalar = vetor_ab[0] * vetor_vertical[0] + vetor_ab[1] * vetor_vertical[1] # (u.v)
    norma_ab = math.sqrt(vetor_ab[0]**2 + vetor_ab[1]**2) # (|u|)
    norma_v_vertical = math.sqrt(vetor_vertical[0]**2 + vetor_vertical[1]**2) # (|v|)

    cos_angulo = produto_escalar / (norma_ab * norma_v_vertical) # cos θ 
    angulo_radianos = math.acos(cos_angulo) # arcos(cos(θ))
    angulo_graus = math.degrees(angulo_radianos) # Porque a função acos retorna em radiano

    return round(angulo_graus, 2) # Retorna apenas duas casas decimais

FUNÇÃO QUE POSCIONA LEGENDA NA TELA

In [4]:
def funcao_texto(texto, cor, img, coord1,coord2):
   '''
   Função que posiciona a legenda na tela de exibição do mediapipe.

   Argumentos da função:
   texto => texto que será exibido na tela
   cor => cor do retangulo que ficará atrás do texto
   img => próprio da função
   coord1 e coord2 => define posicionamento do texto
   '''

   # Definição das propriedades do texto
   font = cv2.FONT_HERSHEY_SIMPLEX
   font_scale = 0.7 
   thickness = 2

   # Obtendo o tamanho do texto
   (text_width, text_height), baseline = cv2.getTextSize(texto, font, font_scale, thickness)

   # Coordenadas do canto superior esquerdo
   x, y = coord1, coord2

   # Calculando o retângulo com base no tamanho do texto
   top_left = (x, y - text_height - baseline)
   bottom_right = (x + text_width, y + baseline)

   # Desenhando o retângulo
   cv2.rectangle(img, top_left, bottom_right, cor, -1)

   cv2.putText(img, texto, (x, y), font, font_scale, (255, 255, 255), thickness)


FUNÇÃO ENVIA STRING PARA ARDUINO

In [5]:
def envia_string(string):
    arduino.write(f"{'Movimento ' + string}\n".encode())  
        # Função write -> Envia valor ao Arduino em bytes
        # Função encode -> transforma em bytes
    return 

## DICIONÁRIOS

Dicionário dos braços

In [6]:
dicionario_poses = {
    "cristo redentor": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloBEC <= (90 + 30) and
            (90 - 30) <= anguloBEB <= (90 + 30) and
            (90 - 30) <= anguloBDC <= (90 + 30) and
            (90 - 30) <= anguloBDB <= (90 + 30)
        )
    },
    "maos pro alto": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (180 - 30) <= anguloBDC <= (180 + 30) and
            (180 - 30) <= anguloBDB <= (180 + 30) and
            (180 - 30) <= anguloBEC <= (180 + 30) and
            (180 - 30) <= anguloBEB <= (180 + 30)
        )
    },
    "maos pra baixo": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (0 - 30) <= anguloBDC <= (0 + 30) and
            (0 - 30) <= anguloBDB <= (0 + 30) and
            (0 - 30) <= anguloBEC <= (0 + 30) and
            (0 - 30) <= anguloBEB <= (0 + 30)
        )
    },
    "esquerda cima direita baixo": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (180 - 30) <= anguloBEC <= (180 + 30) and
            (180 - 30) <= anguloBEB <= (180 + 30) and
            (0 - 30) <= anguloBDC <= (0 + 30) and
            (0 - 30) <= anguloBDB <= (0 + 30)
        )
    },
    "direita cima esquerda baixo": {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (180 - 30) <= anguloBDC <= (180 + 30) and
            (180 - 30) <= anguloBDB <= (180 + 30) and
            (0 - 30) <= anguloBEC <= (0 + 30) and
            (0 - 30) <= anguloBEB <= (0 + 30)
        )
    },
    "formato U":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloBDC <= (90 + 30) and
            (180 - 30) <= anguloBDB <= (180 + 30) and
            (90 - 30) <= anguloBEC <= (90 + 30) and
            (180 - 30) <= anguloBEB <= (180 + 30)
        )
    },
    "formato U invertido":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloBDC <= (90 + 30) and
            (0 - 30) <= anguloBDB <= (0 + 30) and
            (90 - 30) <= anguloBEC <= (90 + 30) and
            (0 - 30) <= anguloBEB <= (0 + 30)
        )
    },
    "formato S":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloBDC <= (90 + 30) and
            (0 - 30) <= anguloBDB <= (0 + 30) and
            (90 - 30) <= anguloBEC <= (90 + 30) and
            (180 - 30) <= anguloBEB <= (180 + 30)
        )
    },
    "formato Ƨ":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloBEC <= (90 + 30) and
            (0 - 30) <= anguloBEB <= (0 + 30) and
            (90 - 30) <= anguloBDC <= (90 + 30) and
            (180 - 30) <= anguloBDB <= (180 + 30)
        )
    },
    "disco move":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (45 - 30) <= anguloBEC <= (45 + 30) and
            (45 - 30) <= anguloBEB <= (45 + 30) and
            (135 - 30) <= anguloBDC <= (135 + 30) and
            (135 - 30) <= anguloBDB <= (135 + 30)
        )
    },
    "disco-disco move":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (45 - 30) <= anguloBDC <= (45 + 30) and
            (45 - 30) <= anguloBDB <= (45 + 30) and
            (135 - 30) <= anguloBEC <= (135 + 30) and
            (135 - 30) <= anguloBEB <= (135 + 30)
        )
    },
    "uzinho":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (50 - 30) <= anguloBDC <= (50 + 30) and
            (180 - 30) <= anguloBDB <= (180 + 30) and
            (50 - 30) <= anguloBEC <= (50 + 30) and
            (180 - 30) <= anguloBEB <= (180 + 30)
        )
    },
    "v":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (135 - 30) <= anguloBDC <= (135 + 30) and
            (135 - 30) <= anguloBDB <= (135 + 30) and
            (135 - 30) <= anguloBEC <= (135 + 30) and
            (135 - 30) <= anguloBEB <= (135 + 30)
        )
    },
     "maos na cabeca":
    {
        "condicao": lambda anguloBEC, anguloBEB, anguloBDC, anguloBDB: (
            (90 - 30) <= anguloBDC <= (90 + 30) and
            (130 - 30) <= anguloBDB <= (130 + 30) and
            (90 - 30) <= anguloBEC <= (90 + 30) and
            (135 - 30) <= anguloBEB <= (130 + 30)
        )
    }, 
}

Dicionário das pernas

In [7]:
dicionario_poses_pernas = {
    "retas": {
        "condicao": lambda angulo_PEC, angulo_PEB, angulo_PDC, angulo_PDB: (
            (0 - 30) <= angulo_PEC <= (0 + 30) and
            (0 - 30) <= angulo_PEB <= (0 + 30) and
            (0 - 30) <= angulo_PDC <= (0 + 30) and
            (0 - 30) <= angulo_PDB <= (0 + 30)
        )
    },
    "dobradas": {
        "condicao": lambda angulo_PEC, angulo_PEB, angulo_PDC, angulo_PDB: (
            (90 - 30) <= angulo_PEC <= (90 + 30) and
            (0 - 30) <= angulo_PEB <= (0 + 30) and
            (90 - 30) <= angulo_PDC <= (90 + 30) and
            (0 - 30) <= angulo_PDB <= (0 + 30)
        )
    },
    "chutes": {
        "condicao": lambda angulo_PEC, angulo_PEB, angulo_PDC, angulo_PDB: (
            (90 - 30) <= angulo_PEC <= (90 + 30) and
            (90 - 30) <= angulo_PEB <= (90 + 30) and
            (90 - 30) <= angulo_PDC <= (90 + 30) and
            (90 - 30) <= angulo_PDB <= (90 + 30)
        )
    },
    "chuta esquerdo": {
        "condicao": lambda angulo_PEC, angulo_PEB, angulo_PDC, angulo_PDB: (
            (90 - 30) <= angulo_PEC <= (90 + 30) and
            (90 - 30) <= angulo_PEB <= (90 + 30) and
            (0 - 30) <= angulo_PDC <= (90 + 30) and
            (0 - 30) <= angulo_PDB <= (90 + 30)
        )
    },
    "chuta direito": {
        "condicao": lambda angulo_PEC, angulo_PEB, angulo_PDC, angulo_PDB: (
            (0 - 30) <= angulo_PEC <= (90 + 30) and
            (0 - 30) <= angulo_PEB <= (90 + 30) and
            (90 - 30) <= angulo_PDC <= (90 + 30) and
            (90 - 30) <= angulo_PDB <= (90 + 30)
        )
    },
    "dobra direita": {
        "condicao": lambda angulo_PEC, angulo_PEB, angulo_PDC, angulo_PDB: (
            (90 - 30) <= angulo_PEC <= (90 + 30) and
            (0 - 30) <= angulo_PEB <= (0 + 30) and
            (0 - 30) <= angulo_PDC <= (90 + 30) and
            (0 - 30) <= angulo_PDB <= (0 + 30)
        )
    },
    "dobra esquerda": {
        "condicao": lambda angulo_PEC, angulo_PEB, angulo_PDC, angulo_PDB: (
            (0 - 30) <= angulo_PEC <= (90 + 30) and
            (0 - 30) <= angulo_PEB <= (0 + 30) and
            (90 - 30) <= angulo_PDC <= (90 + 30) and
            (0 - 30) <= angulo_PDB <= (0 + 30)
        )
    },
}

## CÓDIGO PRINCIPAL

DETECÇÃO DOS MOVIMENTOS POR MIDIAPIPE E INTERFACE GRÁFICA PARA MÚSICA

In [ ]:
# Keep track of current music playing
isPlaying = False

# Initialize Pygame mixer for audio playback
pygame.mixer.init()

# List to store the playlist (paths to songs)
folder_path = 'Gabriel/musicas'
all_files = os.listdir(folder_path)
playlist = [file[:-4] for file in all_files]
print(playlist)
current_song_index = 0  # Index of the currently playing song

with open("Gabriel/data.json", "r", encoding='utf-8') as file:  # will help to track songs timestamps
    songs_json = json.load(file)
print(songs_json.keys())

current_song_info = {}
current_time_s = 0
states_count = 0
current_state_time = 0
hue_brightness_index = 0
beats_index = 0

# Function to toggle fullscreen mode
def toggle_full_screen():
    is_fullscreen = root.attributes('-fullscreen')
    root.attributes('-fullscreen', not is_fullscreen)

# Function to choose a song and update the interface
def choose_song():
    global current_song_index
    global current_song_info
    global isPlaying
    global songs_json
    global current_state_time
    global states_count
    global hue_brightness_index
    global beats_index
    
    song_path = filedialog.askopenfilename(
        initialdir="musicas",  # Default folder 'musicas'
        title="Escolha uma música",
        filetypes=(("MP3 files", "*.mp3"), ("All files", "*.*"))
    )

    if song_path:
        song_name = os.path.basename(song_path)[:-4]

        current_song_info = songs_json[song_name]
        current_state_time = current_song_info["state_time"]
        states_count = 0 
        hue_brightness_index = 0
        beats_index = 0

        song_status.config(text="Ouvindo agora")
        isPlaying = True
        song_label.config(text=f"{song_name}")

        # Update playlist's current song playing
        for index, song in enumerate(playlist):
            if (song == song_name):
                current_song_index = index

        main_frame.grid_forget()  # Hide the main frame
        music_player_frame.grid(row=0, column=0)  # Show the music player frame

        pygame.mixer.music.load(song_path)
        pygame.mixer.music.play(loops=0, start=0.0)

def update_time():
    global current_time_s
    global states_count
    global hue_brightness_index
    global beats_index
    global isPlaying

    if isPlaying:
        try:
            current_time_s = pygame.mixer.music.get_pos() / 1000

            if current_time_s >= states_count * current_state_time and current_time_s <= current_song_info["total_time_length"]:
                states_count += 1 
                hue_brightness_index += 1

            elif current_time_s >= current_song_info["beats"][beats_index]:
                beats_index += 1
        except:
            isPlaying = False

    root.after(1, update_time)

# Function to play/pause the song
def play_pause_song():
    global isPlaying
    if pygame.mixer.music.get_busy():
        pygame.mixer.music.pause()
        play_button.config(text="Tocar")
        song_status.config(text="Música pausada")
        isPlaying = False
    else:
        pygame.mixer.music.unpause()
        play_button.config(text="Pausar")
        song_status.config(text="Ouvindo agora")
        isPlaying = True

def next_song():
    global current_song_index
    global isPlaying
    global current_song_info
    global current_state_time
    global hue_brightness_index
    global beats_index
    global states_count

    current_song_index += 1
    if current_song_index >= len(playlist):
        current_song_index = 0

    pygame.mixer.music.stop()

    next_song = playlist[current_song_index] 
    current_song_info = songs_json[next_song]   
    current_state_time = current_song_info["state_time"]
    hue_brightness_index = 0
    beats_index = 0
    states_count = 0

    play_button.config(text="Pausar")
    song_status.config(text="Ouvindo agora")
    song_label.config(text=f"{next_song}")

    next_song_path = f"./{folder_path}/{next_song}.mp3"

    pygame.mixer.music.load(next_song_path)
    pygame.mixer.music.play(loops=0, start=0.0)
    isPlaying = True

def previous_song():
    global current_song_index
    global isPlaying
    global current_song_info
    global current_state_time
    global hue_brightness_index
    global beats_index
    global states_count

    current_song_index -= 1
    if current_song_index < 0:
        current_song_index = len(playlist) - 1

    pygame.mixer.music.stop()

    previous_song = playlist[current_song_index]    
    current_song_info = songs_json[previous_song]   
    current_state_time = 0
    hue_brightness_index = 0
    beats_index = 0
    states_count = 0

    play_button.config(text="Pausar")
    song_status.config(text="Ouvindo agora")
    song_label.config(text=f"{previous_song}")

    previous_song_path = f"./{folder_path}/{previous_song}.mp3"

    pygame.mixer.music.load(previous_song_path)
    pygame.mixer.music.play(loops=0, start=0.0)
    isPlaying = True

def stop_song():
    global isPlaying

    pygame.mixer.music.stop()
    play_button.config(text="Pausar")
    isPlaying = False
    music_player_frame.grid_forget()
    main_frame.grid(row=0, column=0)  

# Create the main window
root = tk.Tk()
root.attributes('-fullscreen', False)

# Create a main frame
main_frame = ttk.Frame(root, padding=10)
main_frame.grid()

# Title Label
ttk.Label(main_frame, text="ROBÔ DANÇANTE").grid(column=3, row=3)

# Button to choose the song
choose_song_button = ttk.Button(main_frame, text="Escolher música", command=choose_song)
choose_song_button.grid(column=3, row=4)

# Button to toggle fullscreen
full_screen_button = ttk.Button(main_frame, text="Tela cheia", command=toggle_full_screen)
full_screen_button.grid(column=3, row=5)

# Button to exit the program
exit_button = ttk.Button(main_frame, text="Sair", command=root.destroy)
exit_button.grid(column=3, row=6)

# Create the music player frame (initially hidden)
music_player_frame = ttk.Frame(root, padding=10)

song_status = ttk.Label(music_player_frame)
song_status.grid(column=1, row=0)

song_label = ttk.Label(music_player_frame)  
song_label.grid(column=1, row=1)

next_button = ttk.Button(music_player_frame, text="Anterior", command=previous_song)
next_button.grid(column=1, row=2)

play_button = ttk.Button(music_player_frame, text="Pausar", command=play_pause_song)
play_button.grid(column=1, row=3)

next_button = ttk.Button(music_player_frame, text="Próxima", command=next_song)
next_button.grid(column=1, row=4)

stop_button = ttk.Button(music_player_frame, text="Encerrar player", command=stop_song)
stop_button.grid(column=1, row=5)

full_screen_button_music = ttk.Button(music_player_frame, text="Tela cheia", command=toggle_full_screen)
full_screen_button_music.grid(column=1, row=6)

exit_button_music = ttk.Button(music_player_frame, text="Sair", command=root.destroy)
exit_button_music.grid(column=1, row=7)

# Function to start the video processing (MediaPipe + OpenCV)
def start_video_processing():
    video = cv2.VideoCapture(0)
    if not video.isOpened():
        print("Erro: Não foi possível acessar a câmera.")
        exit()

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_tracking_confidence=0.5, min_detection_confidence=0.5)
    mp_draw = mp.solutions.drawing_utils

    while True:
            success, img = video.read()

            # Verifica se a captura foi bem-sucedida e se a imagem não é None
            if not success or img is None:
                print("Erro: Não foi possível capturar a imagem da câmera.")
                break

            # Redimensiona a imagem para garantir que seja processada corretamente
            img = cv2.resize(img, (640, 480))

            # Converte a imagem para RGB para o MediaPipe
            videoRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Processa a imagem com MediaPipe Pose
            results = pose.process(videoRGB)

            # Obtém os landmarks do corpo detectado
            if results.pose_landmarks:
                
                # Desenha os landmarks na imagem
                mp_draw.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS) 

                h, w, _ = img.shape
                landmarks = results.pose_landmarks.landmark

                #-----------------------------------------------------------------------------------------------------------------------------------

                # Calcula as coordenadas dos pontos específicos

                # Pé Direito
                peDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y * h)
                peDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x * w)
                peDZ = int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].z)

                # Pé Esquerdo
                peEY = int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y * h)
                peEX = int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x * w)
                peEZ = int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].z)

                # Mão direita
                maoDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX].y * h)
                maoDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX].x * w)
                maoDZ = int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX].z)
                # Mão esquerda
                maoEY = int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX].y * h)
                maoEX = int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX].x * w)
                maoEZ = int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX].z)


                # Torso direito
                torsoDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x * w)
                torsoDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y * h)

                
                # Torso Esquerdo
                torsoEX = int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * w)
                torsoEY = int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * h)

                # Ombro Direito
                ombroDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w)
                ombroDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h)

                # Ombro Esquerdo
                ombroEX = int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w)
                ombroEY = int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h)

                # Cotovelo Direito
                cotoveloDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x * w)
                cotoveloDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y * h)

                # Cotovelo Esquerdo
                cotoveloEX = int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x * w)
                cotoveloEY = int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y * h)

                # Joelho Direito
                joelhoDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x * w)
                joelhoDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y * h)

                # Joelho Esquerdo
                joelhoEX = int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x * w)
                joelhoEY = int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y * h)


                #-----------------------------------------------------------------------------------------------------------------------------------


                # Calcula os ângulos e exibe (CALCULO DE ANGULO APENAS PARA EXIBIÇÃO: Entre vetor e vetor vertical)
                
                
                # Primeira linha: calcula angulo
                # Segunda linha: exibe angulo

                # Perna direita (baixo)
                anguloPDB = calcula_angulo(peDX,peDY,joelhoDX,joelhoDY)
                cv2.putText(img, str(anguloPDB), (int((peDX+joelhoDX)/2)-40, int((peDY+joelhoDY)/2)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                # Perna direita (cima)
                anguloPDC = calcula_angulo(joelhoDX,joelhoDY,torsoDX,torsoDY)
                cv2.putText(img, str(anguloPDC), (int((joelhoDX+torsoDX)/2)-40, int((joelhoDY+torsoDY)/2)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                    # Perna esquerda (baixo)
                anguloPEB = calcula_angulo(peEX,peEY,joelhoEX,joelhoEY)
                cv2.putText(img, str(anguloPEB), (int((peEX+joelhoEX)/2)+40, int((peEY+joelhoEY)/2)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                # Perna esquerda (cima)
                anguloPEC = calcula_angulo(joelhoEX,joelhoEY,torsoEX,torsoEY)
                cv2.putText(img, str(anguloPEC), (int((joelhoEX+torsoEX)/2)+40, int((joelhoEY+torsoEY)/2)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)


                # Braço (cima) direito
                anguloBDC = calcula_angulo(cotoveloDX,cotoveloDY,ombroDX,ombroDY)
                cv2.putText(img, str(anguloBDC), (int((cotoveloDX+ombroDX)/2)-40, int((cotoveloDY+ombroDY)/2)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                
                # Braço (baixo) direito
                anguloBDB = calcula_angulo(maoDX,maoDY,cotoveloDX,cotoveloDY)
                cv2.putText(img, str(anguloBDB), (int((cotoveloDX + maoDX) / 2)-40, int((cotoveloDY + maoDY) / 2)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                

                # Braço (cima) esquerdo
                anguloBEC = calcula_angulo(cotoveloEX,cotoveloEY,ombroEX,ombroEY)
                cv2.putText(img, str(anguloBEC), (int((cotoveloEX+ombroEX)/2)+40, int((cotoveloEY+ombroEY)/2)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                
                # Braço (baixo) esquerdo
                anguloBEB = calcula_angulo(maoEX,maoEY,cotoveloEX,cotoveloEY)
                cv2.putText(img, str(anguloBEB), (int((cotoveloEX + maoEX) / 2)+40, int((cotoveloEY + maoEY) / 2)), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                
                
                #-----------------------------------------------------------------------------------------------------------------------------------


                # Exibição visual

                texto = str(peDX) + "," + str(peDY) + ","+ str(peDY) 
                cv2.putText(img, texto, (peDX,peDY), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                
                
                texto = str(maoDX) + "," + str(maoDY) + ","+ str(maoDZ) 
                cv2.putText(img, texto, (maoDX,maoDY), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                

                #-----------------------------------------------------------------------------------------------------------------------------------


                
                #-----------------------------------------------------------------------------------------------------------------------------------

                # Percorrer o dicionário e ver se a pessoa está fazendo uma das poses dos BRAÇOS

                for movimento, angulos in dicionario_poses.items():
                    if angulos["condicao"](anguloBEC, anguloBEB, anguloBDC, anguloBDB):
                        cor = (255, 105, 180)
                        texto = movimento
                        coord1 = 10
                        coord2= 300
                        funcao_texto(texto, cor, img, coord1, coord2)
                        print(texto)
                        #envia_string("Movimento " + texto) #OBS: Vai ter que diferencia movimento de perna e braço, tem que ver o codigo do Bruno
                        #BACALHAU

                #-----------------------------------------------------------------------------------------------------------------------------------


                # Percorrer o dicionário e ver se a pessoa está fazendo uma das poses das PERNAS

                for movimento, angulos in dicionario_poses_pernas.items():
                    if angulos["condicao"](anguloPEC, anguloPEB, anguloPDC, anguloPDB):
                        cor = (255, 105, 180)
                        texto = movimento
                        coord1 = 10
                        coord2= 250
                        funcao_texto(texto, cor, img, coord1, coord2)
                        #envia_string("Movimento " + texto) # Adicionei a string movimento para ser coerente com o código do Bruno    
                        #BACALHAU


            #-----------------------------------------------------------------------------------------------------------------------------------

            # Exibição do vídeo em tempo real com os landmarks
            cv2.imshow('Resultado', img)

            # Verifica se a tecla 'q' foi pressionada para sair do loop
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    video.release()
    cv2.destroyAllWindows()

# Run video processing in a separate thread
video_thread = threading.Thread(target=start_video_processing)
video_thread.start()

# Run the Tkinter main loop
update_time()
root.mainloop()


["Bee Gees - Stayin' Alive (Official Music Video)"]
dict_keys(["Bee Gees - Stayin' Alive (Official Music Video)", 'Daft Punk - One More Time (Official Video)', 'David Guetta - Titanium ft. Sia (Official Video)', 'George Michael - Careless Whisper (Official Video) [ ezmp3.cc ]', 'Iron Maiden - The Trooper. HQ audio.', 'Jamily A Fé Faz o Herói', 'Marília Mendonça - Leão - Decretos Reais', 'Oh Happy Day', 'Queen - Another One Bites The Dust (Official Lyric Video)', 'Queen - We Will Rock You (Official Video)', 'Queen – Bohemian Rhapsody (Official Video Remastered)', "Racionais Mc's - A Vida é Desafio", 'Sandra De Sá - Solidão (Áudio Oficial)', 'Smack That (Clean)', 'Snoop Dogg - Beautiful (Official Music Video) ft. Pharrell Williams', 'Sorriso Maroto - Ainda Gosto de Você', 'Tim Maia - Ela Partiu', 'Travis Scott - SICKO MODE', 'We will rock you - Evian Waterboy'])


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\gabri\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\gabri\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 861, in callit
    func(*args)
  File "C:\Users\gabri\AppData\Local\Temp\ipykernel_18404\2033554745.py", line 33, in process_video_frame
    img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
          ^^^^^^^^^^^^^^^
AttributeError: type object 'Image' has no attribute 'fromarray'


KeyboardInterrupt: 

maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
maos pra baixo
disco move
disco move
disco move
disco move
disco move
disco move
disco move
disco move
disco move
disco move
uzinho
uzinho
uzinho
maos pra baixo
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
maos pra baixo
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho


In [20]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import pygame.mixer
import os
import json
import threading
import cv2
from PIL import Image, ImageTk
import mediapipe as mp

# Keep track of current music playing
isPlaying = False
pygame.mixer.init()
folder_path = 'Gabriel/musicas'
all_files = os.listdir(folder_path)
playlist = [file[:-4] for file in all_files]
current_song_index = 0

with open("Gabriel/data.json", "r", encoding='utf-8') as file:  # will help to track songs timestamps
    songs_json = json.load(file)

current_song_info = {}
current_time_s = 0
states_count = 0
current_state_time = 0
hue_brightness_index = 0
beats_index = 0

# Function to toggle fullscreen mode
def toggle_full_screen():
    is_fullscreen = root.attributes('-fullscreen')
    root.attributes('-fullscreen', not is_fullscreen)

# Function to choose a song and update the interface
def choose_song():
    global current_song_index
    global current_song_info
    global isPlaying
    global songs_json
    global current_state_time
    global states_count
    global hue_brightness_index
    global beats_index
    
    song_path = filedialog.askopenfilename(
        initialdir="musicas",  # Default folder 'musicas'
        title="Escolha uma música",
        filetypes=(("MP3 files", "*.mp3"), ("All files", "*.*"))
    )

    if song_path:
        song_name = os.path.basename(song_path)[:-4]

        current_song_info = songs_json[song_name]
        current_state_time = current_song_info["state_time"]
        states_count = 0 
        hue_brightness_index = 0
        beats_index = 0

        song_status.config(text="Ouvindo agora")
        isPlaying = True
        song_label.config(text=f"{song_name}")

        for index, song in enumerate(playlist):
            if song == song_name:
                current_song_index = index

        main_frame.grid_forget()  # Hide the main frame
        music_player_frame.grid(row=0, column=0)  # Show the music player frame

        pygame.mixer.music.load(song_path)
        pygame.mixer.music.play(loops=0, start=0.0)

def update_time():
    global current_time_s
    global states_count
    global hue_brightness_index
    global beats_index
    global isPlaying

    if isPlaying:
        try:
            current_time_s = pygame.mixer.music.get_pos() / 1000

            if current_time_s >= states_count * current_state_time and current_time_s <= current_song_info["total_time_length"]:
                states_count += 1 
                hue_brightness_index += 1

            elif current_time_s >= current_song_info["beats"][beats_index]:
                beats_index += 1
        except:
            isPlaying = False

    root.after(1, update_time)

def play_pause_song():
    global isPlaying
    if pygame.mixer.music.get_busy():
        pygame.mixer.music.pause()
        play_button.config(text="Tocar")
        song_status.config(text="Música pausada")
        isPlaying = False
    else:
        pygame.mixer.music.unpause()
        play_button.config(text="Pausar")
        song_status.config(text="Ouvindo agora")
        isPlaying = True

def next_song():
    global current_song_index
    global isPlaying
    global current_song_info
    global current_state_time
    global hue_brightness_index
    global beats_index
    global states_count

    current_song_index += 1
    if current_song_index >= len(playlist):
        current_song_index = 0

    pygame.mixer.music.stop()

    next_song = playlist[current_song_index] 
    current_song_info = songs_json[next_song]   
    current_state_time = current_song_info["state_time"]
    hue_brightness_index = 0
    beats_index = 0
    states_count = 0

    play_button.config(text="Pausar")
    song_status.config(text="Ouvindo agora")
    song_label.config(text=f"{next_song}")

    next_song_path = f"./{folder_path}/{next_song}.mp3"

    pygame.mixer.music.load(next_song_path)
    pygame.mixer.music.play(loops=0, start=0.0)
    isPlaying = True

def previous_song():
    global current_song_index
    global isPlaying
    global current_song_info
    global current_state_time
    global hue_brightness_index
    global beats_index
    global states_count

    current_song_index -= 1
    if current_song_index < 0:
        current_song_index = len(playlist) - 1

    pygame.mixer.music.stop()

    previous_song = playlist[current_song_index]    
    current_song_info = songs_json[previous_song]   
    current_state_time = 0
    hue_brightness_index = 0
    beats_index = 0
    states_count = 0

    play_button.config(text="Pausar")
    song_status.config(text="Ouvindo agora")
    song_label.config(text=f"{previous_song}")

    previous_song_path = f"./{folder_path}/{previous_song}.mp3"

    pygame.mixer.music.load(previous_song_path)
    pygame.mixer.music.play(loops=0, start=0.0)
    isPlaying = True

def stop_song():
    global isPlaying

    pygame.mixer.music.stop()
    play_button.config(text="Pausar")
    isPlaying = False
    music_player_frame.grid_forget()
    main_frame.grid(row=0, column=0)  

# Create the main window
root = tk.Tk()
root.attributes('-fullscreen', False)

# Create a main frame
main_frame = ttk.Frame(root, padding=10)
main_frame.grid()

# Title Label
ttk.Label(main_frame, text="ROBÔ DANÇANTE").grid(column=3, row=3)

# Button to choose the song
choose_song_button = ttk.Button(main_frame, text="Escolher música", command=choose_song)
choose_song_button.grid(column=3, row=4)

# Button to toggle fullscreen
full_screen_button = ttk.Button(main_frame, text="Tela cheia", command=toggle_full_screen)
full_screen_button.grid(column=3, row=5)

# Button to exit the program
exit_button = ttk.Button(main_frame, text="Sair", command=root.destroy)
exit_button.grid(column=3, row=6)

# Create the music player frame (initially hidden)
music_player_frame = ttk.Frame(root, padding=10)

song_status = ttk.Label(music_player_frame)
song_status.grid(column=1, row=0)

song_label = ttk.Label(music_player_frame)  
song_label.grid(column=1, row=1)

next_button = ttk.Button(music_player_frame, text="Anterior", command=previous_song)
next_button.grid(column=1, row=2)

play_button = ttk.Button(music_player_frame, text="Pausar", command=play_pause_song)
play_button.grid(column=1, row=3)

next_button = ttk.Button(music_player_frame, text="Próxima", command=next_song)
next_button.grid(column=1, row=4)

stop_button = ttk.Button(music_player_frame, text="Encerrar player", command=stop_song)
stop_button.grid(column=1, row=5)

full_screen_button_music = ttk.Button(music_player_frame, text="Tela cheia", command=toggle_full_screen)
full_screen_button_music.grid(column=1, row=6)

exit_button_music = ttk.Button(music_player_frame, text="Sair", command=root.destroy)
exit_button_music.grid(column=1, row=7)

# Function to start the video processing (MediaPipe + OpenCV)
def start_video_processing():
    video = cv2.VideoCapture(0)
    if not video.isOpened():
        print("Erro: Não foi possível acessar a câmera.")
        exit()

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_tracking_confidence=0.5, min_detection_confidence=0.5)
    mp_draw = mp.solutions.drawing_utils

    # Tkinter label for displaying video
    video_label = ttk.Label(root)
    video_label.grid(row=0, column=1)

    def update_video():
        ret, frame = video.read()
        if not ret:
            print("Falha ao capturar o vídeo.")
            return

        # Process frame with MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(frame_rgb)

        # Draw landmarks on the frame
        if result.pose_landmarks:
            mp_draw.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Convert the frame to Image for Tkinter
        frame_img = Image.fromarray(frame)
        frame_tk = ImageTk.PhotoImage(frame_img)

        # Update the image on the Tkinter label
        video_label.img = frame_tk  # Keep a reference to avoid garbage collection
        video_label.configure(image=frame_tk)

        # Call the update function again after 10ms
        root.after(10, update_video)

    update_video()

# Start the video processing in a separate thread to not block the Tkinter main loop
thread = threading.Thread(target=start_video_processing, daemon=True)
thread.start()

# Start the Tkinter main loop
update_time()
root.mainloop()


EXIBE AMBOS NO TKINTER

In [15]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import pygame.mixer
import os
import json
import threading
import cv2
from PIL import Image, ImageTk
import mediapipe as mp

# Keep track of current music playing
isPlaying = False
pygame.mixer.init()
folder_path = 'Gabriel/musicas'
all_files = os.listdir(folder_path)
playlist = [file[:-4] for file in all_files]
current_song_index = 0

with open("Gabriel/data.json", "r", encoding='utf-8') as file:  # will help to track songs timestamps
    songs_json = json.load(file)

current_song_info = {}
current_time_s = 0
states_count = 0
current_state_time = 0
hue_brightness_index = 0
beats_index = 0

# Function to toggle fullscreen mode
def toggle_full_screen():
    is_fullscreen = root.attributes('-fullscreen')
    root.attributes('-fullscreen', not is_fullscreen)

# Function to choose a song and update the interface
def choose_song():
    global current_song_index
    global current_song_info
    global isPlaying
    global songs_json
    global current_state_time
    global states_count
    global hue_brightness_index
    global beats_index
    
    song_path = filedialog.askopenfilename(
        initialdir="musicas",  # Default folder 'musicas'
        title="Escolha uma música",
        filetypes=(("MP3 files", "*.mp3"), ("All files", "*.*"))
    )

    if song_path:
        song_name = os.path.basename(song_path)[:-4]

        current_song_info = songs_json[song_name]
        current_state_time = current_song_info["state_time"]
        states_count = 0 
        hue_brightness_index = 0
        beats_index = 0

        song_status.config(text="Ouvindo agora")
        isPlaying = True
        song_label.config(text=f"{song_name}")

        for index, song in enumerate(playlist):
            if song == song_name:
                current_song_index = index

        main_frame.grid_forget()  # Hide the main frame
        music_player_frame.grid(row=0, column=0)  # Show the music player frame

        pygame.mixer.music.load(song_path)
        pygame.mixer.music.play(loops=0, start=0.0)

def update_time():
    global current_time_s
    global states_count
    global hue_brightness_index
    global beats_index
    global isPlaying

    if isPlaying:
        try:
            current_time_s = pygame.mixer.music.get_pos() / 1000

            if current_time_s >= states_count * current_state_time and current_time_s <= current_song_info["total_time_length"]:
                states_count += 1 
                hue_brightness_index += 1

            elif current_time_s >= current_song_info["beats"][beats_index]:
                beats_index += 1
        except:
            isPlaying = False

    root.after(1, update_time)

def play_pause_song():
    global isPlaying
    if pygame.mixer.music.get_busy():
        pygame.mixer.music.pause()
        play_button.config(text="Tocar")
        song_status.config(text="Música pausada")
        isPlaying = False
    else:
        pygame.mixer.music.unpause()
        play_button.config(text="Pausar")
        song_status.config(text="Ouvindo agora")
        isPlaying = True

def next_song():
    global current_song_index
    global isPlaying
    global current_song_info
    global current_state_time
    global hue_brightness_index
    global beats_index
    global states_count

    current_song_index += 1
    if current_song_index >= len(playlist):
        current_song_index = 0

    pygame.mixer.music.stop()

    next_song = playlist[current_song_index] 
    current_song_info = songs_json[next_song]   
    current_state_time = current_song_info["state_time"]
    hue_brightness_index = 0
    beats_index = 0
    states_count = 0

    play_button.config(text="Pausar")
    song_status.config(text="Ouvindo agora")
    song_label.config(text=f"{next_song}")

    next_song_path = f"./{folder_path}/{next_song}.mp3"

    pygame.mixer.music.load(next_song_path)
    pygame.mixer.music.play(loops=0, start=0.0)
    isPlaying = True

def previous_song():
    global current_song_index
    global isPlaying
    global current_song_info
    global current_state_time
    global hue_brightness_index
    global beats_index
    global states_count

    current_song_index -= 1
    if current_song_index < 0:
        current_song_index = len(playlist) - 1

    pygame.mixer.music.stop()

    previous_song = playlist[current_song_index]    
    current_song_info = songs_json[previous_song]   
    current_state_time = 0
    hue_brightness_index = 0
    beats_index = 0
    states_count = 0

    play_button.config(text="Pausar")
    song_status.config(text="Ouvindo agora")
    song_label.config(text=f"{previous_song}")

    previous_song_path = f"./{folder_path}/{previous_song}.mp3"

    pygame.mixer.music.load(previous_song_path)
    pygame.mixer.music.play(loops=0, start=0.0)
    isPlaying = True

def stop_song():
    global isPlaying

    pygame.mixer.music.stop()
    play_button.config(text="Pausar")
    isPlaying = False
    music_player_frame.grid_forget()
    main_frame.grid(row=0, column=0)  

# Create the main window
root = tk.Tk()
root.attributes('-fullscreen', False)

# Create a main frame
main_frame = ttk.Frame(root, padding=10)
main_frame.grid()

# Title Label
ttk.Label(main_frame, text="ROBÔ DANÇANTE").grid(column=3, row=3)

# Button to choose the song
choose_song_button = ttk.Button(main_frame, text="Escolher música", command=choose_song)
choose_song_button.grid(column=3, row=4)

# Button to toggle fullscreen
full_screen_button = ttk.Button(main_frame, text="Tela cheia", command=toggle_full_screen)
full_screen_button.grid(column=3, row=5)

# Button to exit the program
exit_button = ttk.Button(main_frame, text="Sair", command=root.destroy)
exit_button.grid(column=3, row=6)

# Create the music player frame (initially hidden)
music_player_frame = ttk.Frame(root, padding=10)

song_status = ttk.Label(music_player_frame)
song_status.grid(column=1, row=0)

song_label = ttk.Label(music_player_frame)  
song_label.grid(column=1, row=1)

next_button = ttk.Button(music_player_frame, text="Anterior", command=previous_song)
next_button.grid(column=1, row=2)

play_button = ttk.Button(music_player_frame, text="Pausar", command=play_pause_song)
play_button.grid(column=1, row=3)

next_button = ttk.Button(music_player_frame, text="Próxima", command=next_song)
next_button.grid(column=1, row=4)

stop_button = ttk.Button(music_player_frame, text="Encerrar player", command=stop_song)
stop_button.grid(column=1, row=5)

full_screen_button_music = ttk.Button(music_player_frame, text="Tela cheia", command=toggle_full_screen)
full_screen_button_music.grid(column=1, row=6)

exit_button_music = ttk.Button(music_player_frame, text="Sair", command=root.destroy)
exit_button_music.grid(column=1, row=7)

# Function to start the video processing (MediaPipe + OpenCV)
def start_video_processing():
    video = cv2.VideoCapture(0)
    if not video.isOpened():
        print("Erro: Não foi possível acessar a câmera.")
        exit()

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_tracking_confidence=0.5, min_detection_confidence=0.5)
    mp_draw = mp.solutions.drawing_utils

    # Tkinter label for displaying video
    video_label = ttk.Label(root)
    video_label.grid(row=0, column=1)

    def update_video():
        ret, frame = video.read()
        if not ret:
            print("Falha ao capturar o vídeo.")
            return

        # Process frame with MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(frame_rgb)
        
        # Draw landmarks on the frame
        if result.pose_landmarks:
            mp_draw.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Convert the frame to Image for Tkinter
        frame_img = Image.fromarray(frame)
        frame_tk = ImageTk.PhotoImage(frame_img)

        # Update the image on the Tkinter label
        video_label.img = frame_tk  # Keep a reference to avoid garbage collection
        video_label.configure(image=frame_tk)

        # Call the update function again after 10ms
        root.after(10, update_video)

    update_video()

# Start the video processing in a separate thread to not block the Tkinter main loop
thread = threading.Thread(target=start_video_processing, daemon=True)
thread.start()

# Start the Tkinter main loop
update_time()
root.mainloop()


EXIBIÇÃO DE AMBOS NO TKINTER COM COR

In [13]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import pygame.mixer
import os
import json
import threading
import cv2
from PIL import Image, ImageTk
import mediapipe as mp

# Keep track of current music playing
isPlaying = False
pygame.mixer.init()
folder_path = 'Gabriel/musicas'
all_files = os.listdir(folder_path)
playlist = [file[:-4] for file in all_files]
current_song_index = 0

with open("Gabriel/data.json", "r", encoding='utf-8') as file:  # will help to track songs timestamps
    songs_json = json.load(file)

current_song_info = {}
current_time_s = 0
states_count = 0
current_state_time = 0
hue_brightness_index = 0
beats_index = 0

# Function to toggle fullscreen mode
def toggle_full_screen():
    is_fullscreen = root.attributes('-fullscreen')
    root.attributes('-fullscreen', not is_fullscreen)

# Function to choose a song and update the interface
def choose_song():
    global current_song_index
    global current_song_info
    global isPlaying
    global songs_json
    global current_state_time
    global states_count
    global hue_brightness_index
    global beats_index
    
    song_path = filedialog.askopenfilename(
        initialdir="musicas",  # Default folder 'musicas'
        title="Escolha uma música",
        filetypes=(("MP3 files", "*.mp3"), ("All files", "*.*"))
    )

    if song_path:
        song_name = os.path.basename(song_path)[:-4]

        current_song_info = songs_json[song_name]
        current_state_time = current_song_info["state_time"]
        states_count = 0 
        hue_brightness_index = 0
        beats_index = 0

        song_status.config(text="Ouvindo agora")
        isPlaying = True
        song_label.config(text=f"{song_name}")

        for index, song in enumerate(playlist):
            if song == song_name:
                current_song_index = index

        main_frame.grid_forget()  # Hide the main frame
        music_player_frame.grid(row=0, column=0)  # Show the music player frame

        pygame.mixer.music.load(song_path)
        pygame.mixer.music.play(loops=0, start=0.0)

def update_time():
    global current_time_s
    global states_count
    global hue_brightness_index
    global beats_index
    global isPlaying

    if isPlaying:
        try:
            current_time_s = pygame.mixer.music.get_pos() / 1000

            if current_time_s >= states_count * current_state_time and current_time_s <= current_song_info["total_time_length"]:
                states_count += 1 
                hue_brightness_index += 1

            elif current_time_s >= current_song_info["beats"][beats_index]:
                beats_index += 1
        except:
            isPlaying = False

    root.after(1, update_time)

def play_pause_song():
    global isPlaying
    if pygame.mixer.music.get_busy():
        pygame.mixer.music.pause()
        play_button.config(text="Tocar")
        song_status.config(text="Música pausada")
        isPlaying = False
    else:
        pygame.mixer.music.unpause()
        play_button.config(text="Pausar")
        song_status.config(text="Ouvindo agora")
        isPlaying = True

def next_song():
    global current_song_index
    global isPlaying
    global current_song_info
    global current_state_time
    global hue_brightness_index
    global beats_index
    global states_count

    current_song_index += 1
    if current_song_index >= len(playlist):
        current_song_index = 0

    pygame.mixer.music.stop()

    next_song = playlist[current_song_index] 
    current_song_info = songs_json[next_song]   
    current_state_time = current_song_info["state_time"]
    hue_brightness_index = 0
    beats_index = 0
    states_count = 0

    play_button.config(text="Pausar")
    song_status.config(text="Ouvindo agora")
    song_label.config(text=f"{next_song}")

    next_song_path = f"./{folder_path}/{next_song}.mp3"

    pygame.mixer.music.load(next_song_path)
    pygame.mixer.music.play(loops=0, start=0.0)
    isPlaying = True

def previous_song():
    global current_song_index
    global isPlaying
    global current_song_info
    global current_state_time
    global hue_brightness_index
    global beats_index
    global states_count

    current_song_index -= 1
    if current_song_index < 0:
        current_song_index = len(playlist) - 1

    pygame.mixer.music.stop()

    previous_song = playlist[current_song_index]    
    current_song_info = songs_json[previous_song]   
    current_state_time = 0
    hue_brightness_index = 0
    beats_index = 0
    states_count = 0

    play_button.config(text="Pausar")
    song_status.config(text="Ouvindo agora")
    song_label.config(text=f"{previous_song}")

    previous_song_path = f"./{folder_path}/{previous_song}.mp3"

    pygame.mixer.music.load(previous_song_path)
    pygame.mixer.music.play(loops=0, start=0.0)
    isPlaying = True

def stop_song():
    global isPlaying

    pygame.mixer.music.stop()
    play_button.config(text="Pausar")
    isPlaying = False
    music_player_frame.grid_forget()
    main_frame.grid(row=0, column=0)  

# Create the main window
root = tk.Tk()
root.attributes('-fullscreen', False)

# Create a main frame
main_frame = ttk.Frame(root, padding=10)
main_frame.grid()

# Title Label
ttk.Label(main_frame, text="ROBÔ DANÇANTE").grid(column=3, row=3)

# Button to choose the song
choose_song_button = ttk.Button(main_frame, text="Escolher música", command=choose_song)
choose_song_button.grid(column=3, row=4)

# Button to toggle fullscreen
full_screen_button = ttk.Button(main_frame, text="Tela cheia", command=toggle_full_screen)
full_screen_button.grid(column=3, row=5)

# Button to exit the program
exit_button = ttk.Button(main_frame, text="Sair", command=root.destroy)
exit_button.grid(column=3, row=6)

# Create the music player frame (initially hidden)
music_player_frame = ttk.Frame(root, padding=10)

song_status = ttk.Label(music_player_frame)
song_status.grid(column=1, row=0)

song_label = ttk.Label(music_player_frame)  
song_label.grid(column=1, row=1)

next_button = ttk.Button(music_player_frame, text="Anterior", command=previous_song)
next_button.grid(column=1, row=2)

play_button = ttk.Button(music_player_frame, text="Pausar", command=play_pause_song)
play_button.grid(column=1, row=3)

next_button = ttk.Button(music_player_frame, text="Próxima", command=next_song)
next_button.grid(column=1, row=4)

stop_button = ttk.Button(music_player_frame, text="Encerrar player", command=stop_song)
stop_button.grid(column=1, row=5)

full_screen_button_music = ttk.Button(music_player_frame, text="Tela cheia", command=toggle_full_screen)
full_screen_button_music.grid(column=1, row=6)

exit_button_music = ttk.Button(music_player_frame, text="Sair", command=root.destroy)
exit_button_music.grid(column=1, row=7)

# Function to start the video processing (MediaPipe + OpenCV)


# def start_video_processing():
#     video = cv2.VideoCapture(0)
#     if not video.isOpened():
#         print("Erro: Não foi possível acessar a câmera.")
#         exit()

#     mp_pose = mp.solutions.pose
#     pose = mp_pose.Pose(min_tracking_confidence=0.5, min_detection_confidence=0.5)
#     mp_draw = mp.solutions.drawing_utils

#     # Tkinter label for displaying video
#     video_label = ttk.Label(root)
#     video_label.grid(row=0, column=1)

#     def update_video():
#         # ret, frame = video.read()
#         # if not ret:
#         #     print("Falha ao capturar o vídeo.")
#         #     return

#         # # Process frame with MediaPipe
#         # frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         # result = pose.process(frame_rgb)
        
#         # # Draw landmarks on the frame
#         # if result.pose_landmarks:
#         #     mp_draw.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

#         # # Convert the frame to Image for Tkinter
#         # frame_img = Image.fromarray(frame)
#         # frame_tk = ImageTk.PhotoImage(frame_img)

#         # # Update the image on the Tkinter label
#         # video_label.img = frame_tk  # Keep a reference to avoid garbage collection
#         # video_label.configure(image=frame_tk)

#         # # Call the update function again after 10ms
#         # root.after(10, update_video)
#         success, img = video.read()

#         # Verifica se a captura foi bem-sucedida e se a imagem não é None
#         if not success or img is None:
#             print("Erro: Não foi possível capturar a imagem da câmera.")
#             break

#         # Redimensiona a imagem para garantir que seja processada corretamente
#         img = cv2.resize(img, (640, 480))

#         # Converte a imagem para RGB para o MediaPipe
#         videoRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#         # Processa a imagem com MediaPipe Pose
#         results = pose.process(videoRGB)

#         # Obtém os landmarks do corpo detectado
#         if results.pose_landmarks:
            
#             # Desenha os landmarks na imagem
#             mp_draw.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS) 

#             h, w, _ = img.shape
#             landmarks = results.pose_landmarks.landmark

#             #-----------------------------------------------------------------------------------------------------------------------------------

#             # Calcula as coordenadas dos pontos específicos

#             # Pé Direito
#             peDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y * h)
#             peDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x * w)
#             peDZ = int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].z)

#             # Pé Esquerdo
#             peEY = int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y * h)
#             peEX = int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x * w)
#             peEZ = int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].z)

#             # Mão direita
#             maoDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX].y * h)
#             maoDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX].x * w)
#             maoDZ = int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX].z)
#             # Mão esquerda
#             maoEY = int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX].y * h)
#             maoEX = int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX].x * w)
#             maoEZ = int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX].z)


#             # Torso direito
#             torsoDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x * w)
#             torsoDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y * h)

            
#             # Torso Esquerdo
#             torsoEX = int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * w)
#             torsoEY = int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * h)

#             # Ombro Direito
#             ombroDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w)
#             ombroDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h)

#             # Ombro Esquerdo
#             ombroEX = int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w)
#             ombroEY = int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h)

#             # Cotovelo Direito
#             cotoveloDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x * w)
#             cotoveloDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y * h)

#             # Cotovelo Esquerdo
#             cotoveloEX = int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x * w)
#             cotoveloEY = int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y * h)

#             # Joelho Direito
#             joelhoDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x * w)
#             joelhoDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y * h)

#             # Joelho Esquerdo
#             joelhoEX = int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x * w)
#             joelhoEY = int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y * h)


#             #-----------------------------------------------------------------------------------------------------------------------------------


#             # Calcula os ângulos e exibe (CALCULO DE ANGULO APENAS PARA EXIBIÇÃO: Entre vetor e vetor vertical)
            
            
#             # Primeira linha: calcula angulo
#             # Segunda linha: exibe angulo

#             # Perna direita (baixo)
#             anguloPDB = calcula_angulo(peDX,peDY,joelhoDX,joelhoDY)
#             cv2.putText(img, str(anguloPDB), (int((peDX+joelhoDX)/2)-40, int((peDY+joelhoDY)/2)), 
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

#             # Perna direita (cima)
#             anguloPDC = calcula_angulo(joelhoDX,joelhoDY,torsoDX,torsoDY)
#             cv2.putText(img, str(anguloPDC), (int((joelhoDX+torsoDX)/2)-40, int((joelhoDY+torsoDY)/2)), 
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

#                 # Perna esquerda (baixo)
#             anguloPEB = calcula_angulo(peEX,peEY,joelhoEX,joelhoEY)
#             cv2.putText(img, str(anguloPEB), (int((peEX+joelhoEX)/2)+40, int((peEY+joelhoEY)/2)), 
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

#             # Perna esquerda (cima)
#             anguloPEC = calcula_angulo(joelhoEX,joelhoEY,torsoEX,torsoEY)
#             cv2.putText(img, str(anguloPEC), (int((joelhoEX+torsoEX)/2)+40, int((joelhoEY+torsoEY)/2)), 
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)


#             # Braço (cima) direito
#             anguloBDC = calcula_angulo(cotoveloDX,cotoveloDY,ombroDX,ombroDY)
#             cv2.putText(img, str(anguloBDC), (int((cotoveloDX+ombroDX)/2)-40, int((cotoveloDY+ombroDY)/2)), 
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
#             # Braço (baixo) direito
#             anguloBDB = calcula_angulo(maoDX,maoDY,cotoveloDX,cotoveloDY)
#             cv2.putText(img, str(anguloBDB), (int((cotoveloDX + maoDX) / 2)-40, int((cotoveloDY + maoDY) / 2)), 
#                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            

#             # Braço (cima) esquerdo
#             anguloBEC = calcula_angulo(cotoveloEX,cotoveloEY,ombroEX,ombroEY)
#             cv2.putText(img, str(anguloBEC), (int((cotoveloEX+ombroEX)/2)+40, int((cotoveloEY+ombroEY)/2)), 
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
#             # Braço (baixo) esquerdo
#             anguloBEB = calcula_angulo(maoEX,maoEY,cotoveloEX,cotoveloEY)
#             cv2.putText(img, str(anguloBEB), (int((cotoveloEX + maoEX) / 2)+40, int((cotoveloEY + maoEY) / 2)), 
#                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            
            
#             #-----------------------------------------------------------------------------------------------------------------------------------


#             # Exibição visual

#             texto = str(peDX) + "," + str(peDY) + ","+ str(peDY) 
#             cv2.putText(img, texto, (peDX,peDY), 
#                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            
#             texto = str(maoDX) + "," + str(maoDY) + ","+ str(maoDZ) 
#             cv2.putText(img, texto, (maoDX,maoDY), 
#                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            

#             #-----------------------------------------------------------------------------------------------------------------------------------


            
#             #-----------------------------------------------------------------------------------------------------------------------------------

#             # Percorrer o dicionário e ver se a pessoa está fazendo uma das poses dos BRAÇOS

#             for movimento, angulos in dicionario_poses.items():
#                 if angulos["condicao"](anguloBEC, anguloBEB, anguloBDC, anguloBDB):
#                     cor = (255, 105, 180)
#                     texto = movimento
#                     coord1 = 10
#                     coord2= 300
#                     funcao_texto(texto, cor, img, coord1, coord2)
#                     print(texto)
#                     #envia_string("Movimento " + texto) #OBS: Vai ter que diferencia movimento de perna e braço, tem que ver o codigo do Bruno
#                     #BACALHAU

#             #-----------------------------------------------------------------------------------------------------------------------------------


#             # Percorrer o dicionário e ver se a pessoa está fazendo uma das poses das PERNAS

#             for movimento, angulos in dicionario_poses_pernas.items():
#                 if angulos["condicao"](anguloPEC, anguloPEB, anguloPDC, anguloPDB):
#                     cor = (255, 105, 180)
#                     texto = movimento
#                     coord1 = 10
#                     coord2= 250
#                     funcao_texto(texto, cor, img, coord1, coord2)
#                     #envia_string("Movimento " + texto) # Adicionei a string movimento para ser coerente com o código do Bruno    
#                     #BACALHAU


#         #-----------------------------------------------------------------------------------------------------------------------------------

#         # Exibição do vídeo em tempo real com os landmarks
#         cv2.imshow('Resultado', img)

#         # Verifica se a tecla 'q' foi pressionada para sair do loop
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     update_video()

# Start the video processing in a separate thread to not block the Tkinter main loop
thread = threading.Thread(target=start_video_processing, daemon=True)
thread.start()

# Start the Tkinter main loop
update_time()
root.mainloop()


uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho
uzinho


TESTE

In [16]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import pygame.mixer
import os
import json
import threading
import cv2
from PIL import Image, ImageTk
import mediapipe as mp

# Keep track of current music playing
isPlaying = False
pygame.mixer.init()
folder_path = 'Gabriel/musicas'
all_files = os.listdir(folder_path)
playlist = [file[:-4] for file in all_files]
current_song_index = 0

with open("Gabriel/data.json", "r", encoding='utf-8') as file:  # will help to track songs timestamps
    songs_json = json.load(file)

current_song_info = {}
current_time_s = 0
states_count = 0
current_state_time = 0
hue_brightness_index = 0
beats_index = 0

# Function to toggle fullscreen mode
def toggle_full_screen():
    is_fullscreen = root.attributes('-fullscreen')
    root.attributes('-fullscreen', not is_fullscreen)

# Function to choose a song and update the interface
def choose_song():
    global current_song_index
    global current_song_info
    global isPlaying
    global songs_json
    global current_state_time
    global states_count
    global hue_brightness_index
    global beats_index
    
    song_path = filedialog.askopenfilename(
        initialdir="musicas",  # Default folder 'musicas'
        title="Escolha uma música",
        filetypes=(("MP3 files", "*.mp3"), ("All files", "*.*"))
    )

    if song_path:
        song_name = os.path.basename(song_path)[:-4]

        current_song_info = songs_json[song_name]
        current_state_time = current_song_info["state_time"]
        states_count = 0 
        hue_brightness_index = 0
        beats_index = 0

        song_status.config(text="Ouvindo agora")
        isPlaying = True
        song_label.config(text=f"{song_name}")

        for index, song in enumerate(playlist):
            if song == song_name:
                current_song_index = index

        main_frame.grid_forget()  # Hide the main frame
        music_player_frame.grid(row=0, column=0)  # Show the music player frame

        pygame.mixer.music.load(song_path)
        pygame.mixer.music.play(loops=0, start=0.0)

def update_time():
    global current_time_s
    global states_count
    global hue_brightness_index
    global beats_index
    global isPlaying

    if isPlaying:
        try:
            current_time_s = pygame.mixer.music.get_pos() / 1000

            if current_time_s >= states_count * current_state_time and current_time_s <= current_song_info["total_time_length"]:
                states_count += 1 
                hue_brightness_index += 1

            elif current_time_s >= current_song_info["beats"][beats_index]:
                beats_index += 1
        except:
            isPlaying = False

    root.after(1, update_time)

def play_pause_song():
    global isPlaying
    if pygame.mixer.music.get_busy():
        pygame.mixer.music.pause()
        play_button.config(text="Tocar")
        song_status.config(text="Música pausada")
        isPlaying = False
    else:
        pygame.mixer.music.unpause()
        play_button.config(text="Pausar")
        song_status.config(text="Ouvindo agora")
        isPlaying = True

def next_song():
    global current_song_index
    global isPlaying
    global current_song_info
    global current_state_time
    global hue_brightness_index
    global beats_index
    global states_count

    current_song_index += 1
    if current_song_index >= len(playlist):
        current_song_index = 0

    pygame.mixer.music.stop()

    next_song = playlist[current_song_index] 
    current_song_info = songs_json[next_song]   
    current_state_time = current_song_info["state_time"]
    hue_brightness_index = 0
    beats_index = 0
    states_count = 0

    play_button.config(text="Pausar")
    song_status.config(text="Ouvindo agora")
    song_label.config(text=f"{next_song}")

    next_song_path = f"./{folder_path}/{next_song}.mp3"

    pygame.mixer.music.load(next_song_path)
    pygame.mixer.music.play(loops=0, start=0.0)
    isPlaying = True

def previous_song():
    global current_song_index
    global isPlaying
    global current_song_info
    global current_state_time
    global hue_brightness_index
    global beats_index
    global states_count

    current_song_index -= 1
    if current_song_index < 0:
        current_song_index = len(playlist) - 1

    pygame.mixer.music.stop()

    previous_song = playlist[current_song_index]    
    current_song_info = songs_json[previous_song]   
    current_state_time = 0
    hue_brightness_index = 0
    beats_index = 0
    states_count = 0

    play_button.config(text="Pausar")
    song_status.config(text="Ouvindo agora")
    song_label.config(text=f"{previous_song}")

    previous_song_path = f"./{folder_path}/{previous_song}.mp3"

    pygame.mixer.music.load(previous_song_path)
    pygame.mixer.music.play(loops=0, start=0.0)
    isPlaying = True

def stop_song():
    global isPlaying

    pygame.mixer.music.stop()
    play_button.config(text="Pausar")
    isPlaying = False
    music_player_frame.grid_forget()
    main_frame.grid(row=0, column=0)  

# Create the main window
root = tk.Tk()
root.attributes('-fullscreen', False)

# Create a main frame
main_frame = ttk.Frame(root, padding=10)
main_frame.grid()

# Title Label
ttk.Label(main_frame, text="ROBÔ DANÇANTE").grid(column=3, row=3)

# Button to choose the song
choose_song_button = ttk.Button(main_frame, text="Escolher música", command=choose_song)
choose_song_button.grid(column=3, row=4)

# Button to toggle fullscreen
full_screen_button = ttk.Button(main_frame, text="Tela cheia", command=toggle_full_screen)
full_screen_button.grid(column=3, row=5)

# Button to exit the program
exit_button = ttk.Button(main_frame, text="Sair", command=root.destroy)
exit_button.grid(column=3, row=6)

# Create the music player frame (initially hidden)
music_player_frame = ttk.Frame(root, padding=10)

song_status = ttk.Label(music_player_frame)
song_status.grid(column=1, row=0)

song_label = ttk.Label(music_player_frame)  
song_label.grid(column=1, row=1)

next_button = ttk.Button(music_player_frame, text="Anterior", command=previous_song)
next_button.grid(column=1, row=2)

play_button = ttk.Button(music_player_frame, text="Pausar", command=play_pause_song)
play_button.grid(column=1, row=3)

next_button = ttk.Button(music_player_frame, text="Próxima", command=next_song)
next_button.grid(column=1, row=4)

stop_button = ttk.Button(music_player_frame, text="Encerrar player", command=stop_song)
stop_button.grid(column=1, row=5)

full_screen_button_music = ttk.Button(music_player_frame, text="Tela cheia", command=toggle_full_screen)
full_screen_button_music.grid(column=1, row=6)

exit_button_music = ttk.Button(music_player_frame, text="Sair", command=root.destroy)
exit_button_music.grid(column=1, row=7)

# Function to start the video processing (MediaPipe + OpenCV)


def start_video_processing():
    video = cv2.VideoCapture(0)
    if not video.isOpened():
        print("Erro: Não foi possível acessar a câmera.")
        exit()

    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_tracking_confidence=0.5, min_detection_confidence=0.5)
    mp_draw = mp.solutions.drawing_utils

    # Tkinter label for displaying video
    video_label = ttk.Label(root)
    video_label.grid(row=0, column=1)

    def update_video():
        ret, frame = video.read()
        if not ret:
            print("Falha ao capturar o vídeo.")
            return

        # Convert BGR to RGB for MediaPipe processing
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(frame_rgb)

        # Verifica se os landmarks foram encontrados
        if result.pose_landmarks:
            # Desenha os landmarks na imagem
            mp_draw.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            h, w, _ = frame.shape
            landmarks = result.pose_landmarks.landmark

            #-----------------------------------------------------------------------------------------------------------------------------------
            # Calcula as coordenadas dos pontos específicos
            # Pé Direito
            peDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y * h)
            peDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x * w)
            peDZ = int(landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].z)

            # Pé Esquerdo
            peEY = int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y * h)
            peEX = int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x * w)
            peEZ = int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].z)

            # Mão direita
            maoDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX].y * h)
            maoDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX].x * w)
            maoDZ = int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX].z)
            # Mão esquerda
            maoEY = int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX].y * h)
            maoEX = int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX].x * w)
            maoEZ = int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX].z)

            # Torso direito
            torsoDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x * w)
            torsoDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y * h)

            # Torso Esquerdo
            torsoEX = int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].x * w)
            torsoEY = int(landmarks[mp_pose.PoseLandmark.LEFT_HIP].y * h)

            # Ombro Direito
            ombroDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x * w)
            ombroDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y * h)

            # Ombro Esquerdo
            ombroEX = int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * w)
            ombroEY = int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * h)

            # Cotovelo Direito
            cotoveloDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x * w)
            cotoveloDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y * h)

            # Cotovelo Esquerdo
            cotoveloEX = int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x * w)
            cotoveloEY = int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y * h)

            # Joelho Direito
            joelhoDX = int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x * w)
            joelhoDY = int(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y * h)

            # Joelho Esquerdo
            joelhoEX = int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x * w)
            joelhoEY = int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y * h)

            #-----------------------------------------------------------------------------------------------------------------------------------
            # Calcula os ângulos e exibe (CALCULO DE ANGULO APENAS PARA EXIBIÇÃO: Entre vetor e vetor vertical)

            # Perna direita (baixo)
            anguloPDB = calcula_angulo(peDX, peDY, joelhoDX, joelhoDY)
            cv2.putText(frame, str(anguloPDB), (int((peDX+joelhoDX)/2)-40, int((peDY+joelhoDY)/2)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Perna direita (cima)
            anguloPDC = calcula_angulo(joelhoDX, joelhoDY, torsoDX, torsoDY)
            cv2.putText(frame, str(anguloPDC), (int((joelhoDX+torsoDX)/2)-40, int((joelhoDY+torsoDY)/2)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Perna esquerda (baixo)
            anguloPEB = calcula_angulo(peEX, peEY, joelhoEX, joelhoEY)
            cv2.putText(frame, str(anguloPEB), (int((peEX+joelhoEX)/2)+40, int((peEY+joelhoEY)/2)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Perna esquerda (cima)
            anguloPEC = calcula_angulo(joelhoEX, joelhoEY, torsoEX, torsoEY)
            cv2.putText(frame, str(anguloPEC), (int((joelhoEX+torsoEX)/2)+40, int((joelhoEY+torsoEY)/2)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Braço (cima) direito
            anguloBDC = calcula_angulo(cotoveloDX, cotoveloDY, ombroDX, ombroDY)
            cv2.putText(frame, str(anguloBDC), (int((cotoveloDX+ombroDX)/2)-40, int((cotoveloDY+ombroDY)/2)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Braço (baixo) direito
            anguloBDB = calcula_angulo(maoDX, maoDY, cotoveloDX, cotoveloDY)
            cv2.putText(frame, str(anguloBDB), (int((cotoveloDX + maoDX) / 2)-40, int((cotoveloDY + maoDY) / 2)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Braço (cima) esquerdo
            anguloBEC = calcula_angulo(cotoveloEX, cotoveloEY, ombroEX, ombroEY)
            cv2.putText(frame, str(anguloBEC), (int((cotoveloEX+ombroEX)/2)+40, int((cotoveloEY+ombroEY)/2)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # Braço (baixo) esquerdo
            anguloBEB = calcula_angulo(maoEX, maoEY, cotoveloEX, cotoveloEY)
            cv2.putText(frame, str(anguloBEB), (int((cotoveloEX + maoEX) / 2)+40, int((cotoveloEY + maoEY) / 2)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            #-----------------------------------------------------------------------------------------------------------------------------------
            # Exibição visual

            texto = str(peDX) + "," + str(peDY) + "," + str(peDZ)
            cv2.putText(frame, texto, (peDX, peDY),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            texto = str(maoDX) + "," + str(maoDY) + "," + str(maoDZ)
            cv2.putText(frame, texto, (maoDX, maoDY),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            #-----------------------------------------------------------------------------------------------------------------------------------
            # Percorrer o dicionário e ver se a pessoa está fazendo uma das poses dos BRAÇOS
            for movimento, angulos in dicionario_poses.items():
                if angulos["condicao"](anguloBEC, anguloBEB, anguloBDC, anguloBDB):
                    cor = (255, 105, 180)
                    texto = movimento
                    coord1 = 10
                    coord2 = 300
                    funcao_texto(texto, cor, frame, coord1, coord2)

            #-----------------------------------------------------------------------------------------------------------------------------------
            # Percorrer o dicionário e ver se a pessoa está fazendo uma das poses das PERNAS
            for movimento, angulos in dicionario_poses_pernas.items():
                if angulos["condicao"](anguloPEC, anguloPEB, anguloPDC, anguloPDB):
                    cor = (255, 105, 180)
                    texto = movimento
                    coord1 = 10
                    coord2 = 250
                    funcao_texto(texto, cor, frame, coord1, coord2)


        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        # Convert the frame to Image for Tkinter
        frame_img = Image.fromarray(frame_bgr)
        frame_tk = ImageTk.PhotoImage(frame_img)

        # Update the image on the Tkinter label
        video_label.img = frame_tk  # Keep a reference to avoid garbage collection
        video_label.configure(image=frame_tk)

        # Call the update function again after 10ms
        root.after(10, update_video)

    update_video()


# Start the video processing in a separate thread to not block the Tkinter main loop
thread = threading.Thread(target=start_video_processing, daemon=True)
thread.start()

# Start the Tkinter main loop
update_time()
root.mainloop()


uzinho
uzinho
uzinho
maos na cabeca
maos na cabeca
maos na cabeca
v
maos na cabeca
v
v
v
v
maos na cabeca
v
maos na cabeca
v
maos na cabeca
v
maos na cabeca
cristo redentor
v
maos na cabeca
v
v
v
v
v
v
v
v
v
v
v
v
maos na cabeca
maos na cabeca
maos na cabeca
maos na cabeca
